# Evaluation Scratch Book
Experiments and stuff to be later removed.

## Imports

In [152]:
import grid2op
from grid2op.utils import ScoreL2RPN2022
from grid2op.Agent import DoNothingAgent
import pandas as pd
from pathlib import Path
import torch as th
import os
import numpy as np

## Settings

In [153]:
env_name = "rte_case14_realistic_val10"
difficulty = 1
episodes = 10
save_path = "/home/l2rpn/data/evaluation/dpop_rte_very_small"
processes = 14

## Initialization

### Environment

In [155]:
env = grid2op.make(env_name, difficulty=difficulty)

### Load Checkpoint
Torch needs the same directory structure to reload the checkpoint so chdir is necessary to reload the checkpoint correctly

In [156]:
os.chdir("/home/l2rpn/pop/pop") 
checkpoint = th.load("/home/l2rpn/data/checkpoint/dpop_rte_1e5_very_small/dpop_rte_1e5_very_small_50.pt")

In [158]:
checkpoint["architecture"]["pop"]["disabled_action_loops_length"] = 1

#### Fix Legacy Checkpoints
Add exploration method to legacy checkpoints

In [127]:
checkpoint["architecture"]["agent"]

{'embedding': {'layers': [{'name': 'attention1',
    'type': 'EGATConv',
    'module': 'dgl',
    'kwargs': {'type': 'EGATConv',
     'in_node_feats': '<node_features>',
     'in_edge_feats': '<edge_features>',
     'out_node_feats': '8',
     'out_edge_feats': '8',
     'num_heads': 2,
     'bias': True}},
   {'name': 'flatten1',
    'type': 'EGATFlatten',
    'module': 'custom_layers',
    'kwargs': {'type': 'EGATFlatten'}},
   {'name': 'conv',
    'type': 'EGATNodeConv',
    'module': 'pop.graph_convolutional_networks.custom_layers',
    'kwargs': {'type': 'EGATNodeConv',
     'in_feats': array(16, dtype=int32),
     'out_feats': '32',
     'bias': True,
     'weight': True,
     'allow_zero_in_degree': True}}]},
 'advantage_stream': {'layers': [{'name': 'linear1',
    'type': 'Linear',
    'module': 'pytorch',
    'kwargs': {'type': 'Linear',
     'in_features': '<embedding_size>',
     'out_features': 16,
     'bias': True}},
   {'name': 'activation1',
    'type': 'ReLU',
    'mod

In [93]:
checkpoint["architecture"]["agent"]["exploration"]["method"]="epsilon_greedy"
checkpoint["architecture"]["manager"]["exploration"]["method"]="epsilon_greedy"
checbkpoint["architecture"]["head_manager"]["exploration"]["method"]="epsilon_greedy"

In [132]:
def _exponential_decay(decay_steps: int, max_val: float, min_val: float, decay: int) -> float:
    return min_val + (max_val - min_val) * np.exp(-1.0 * decay_steps / decay)


In [134]:
for agent in list(checkpoint["agents_state"].keys()):
    cur_agent = checkpoint["agents_state"][agent]
    if cur_agent.get("architecture"):
        cur_agent["architecture"]["exploration"]["method"] = "epsilon_greedy"
        cur_agent["exploration"]= {}
        cur_agent["exploration"]["decay_steps"] = cur_agent["decay_steps"]    
        cur_agent_mem = cur_agent["memory"]
        del cur_agent_mem["capacity"]
        del cur_agent_mem["alpha"]
        cur_agent_mem["architecture"] = checkpoint["architecture"]["agent"]["replay_memory"]
        cur_agent_mem["beta"] = _exponential_decay(cur_agent["decay_steps"], cur_agent_mem["architecture"]["max_beta"], cur_agent_mem["architecture"]["min_beta"], cur_agent_mem["architecture"]["beta_decay"])
checkpoint["agents_state"][2]["architecture"]["exploration"]
checkpoint["agents_state"][2]["exploration"]


{'decay_steps': 53843}

In [137]:
for manager in list(checkpoint["managers_state"].keys()):
        cur_manager = checkpoint["managers_state"][manager][0]
        cur_manager["architecture"]["exploration"]["method"] = "epsilon_greedy"
        cur_manager["exploration"]= {}
        cur_manager["exploration"]["decay_steps"] = cur_manager["decay_steps"]    
        cur_agent_mem = cur_manager["memory"]
        del cur_agent_mem["capacity"]
        del cur_agent_mem["alpha"]
        cur_agent_mem["architecture"] = checkpoint["architecture"]["manager"]["replay_memory"]
        cur_agent_mem["beta"] = _exponential_decay(cur_manager["decay_steps"], cur_agent_mem["architecture"]["max_beta"], cur_agent_mem["architecture"]["min_beta"], cur_agent_mem["architecture"]["beta_decay"])

checkpoint["managers_state"][next(iter(checkpoint["managers_state"].keys()))][0]["architecture"]["exploration"]
checkpoint["managers_state"][list(checkpoint["managers_state"].keys())[4]][0]["exploration"]

{'decay_steps': 0}

In [138]:
checkpoint["head_manager_state"]["architecture"]["exploration"]["method"] = "epsilon_greedy"
checkpoint["head_manager_state"]["exploration"]= {}
checkpoint["head_manager_state"]["exploration"]["decay_steps"] = checkpoint["head_manager_state"]["decay_steps"]    
cur_agent_mem = checkpoint["head_manager_state"]["memory"]
del cur_agent_mem["capacity"]
del cur_agent_mem["alpha"]
cur_agent_mem["architecture"] = checkpoint["architecture"]["head_manager"]["replay_memory"]
cur_agent_mem["beta"] = _exponential_decay(cur_manager["decay_steps"], cur_agent_mem["architecture"]["max_beta"], cur_agent_mem["architecture"]["min_beta"], cur_agent_mem["architecture"]["beta_decay"])

checkpoint["head_manager_state"]["architecture"]["exploration"]
checkpoint["head_manager_state"]["exploration"]

{'decay_steps': 99644}

In [159]:
os.chdir("/home/l2rpn/pop/pop") 
th.save(checkpoint, "/home/l2rpn/data/checkpoint/dpop_rte_1e5_very_small/dpop_rte_1e5_very_small_51.pt")

{'decay_steps': 42139}

### Do Nothing Agent

In [28]:
agent = DoNothingAgent(env.action_space)

## Score Computation

In [29]:
score = ScoreL2RPN2022(env,nb_scenario=episodes, verbose=1)

In [30]:
agent_score = score.get(agent, nb_process = processes)
agent_score_df = pd.DataFrame(agent_score).transpose()
agent_score_df.columns = ["all_scores", "ts_survived", "total_ts"]

Starts the evaluation of the agent
Start the evaluation of the scores


In [31]:
agent_score_df

,all_scores,ts_survived,total_ts
0,0.0,1097.0,8064.0
1,0.0,3399.0,8064.0
2,0.0,1099.0,8064.0
3,0.0,377.0,8064.0
4,0.0,809.0,8064.0
...,...,...,...
95,0.0,521.0,8064.0
96,0.0,511.0,8064.0
97,0.0,1102.0,8064.0
98,0.0,522.0,8064.0


In [32]:
agent_score_df.to_csv(Path(save_path, "l2rpn_2022_score_" + str(difficulty) + ".csv"))

## Score Evaluation

In [3]:
scores = []
d for d in [0, 1, 2, "competition"]:
    scores.append(pd.load_csv(Path(save_path, "l2rpn_2022_score_" + str(d) + ".csv")))

SyntaxError: invalid syntax (2684874982.py, line 2)